In [9]:
from tokenizer import Tokenizer

tokenizer = Tokenizer()


In [10]:
model = tokenizer.load_model()
model

In [12]:
a = model.encode('I am a president')
a

[15953, 1, 373, 8, 5140]

In [14]:
import torch.nn.functional as F
import torch
onehot = F.one_hot(torch.tensor(a), 16000)
onehot


tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 1, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]])

In [16]:
onehot.shape

torch.Size([5, 16000])

In [7]:
import torch
inputs = torch.rand(64, 20).long()
inputs

tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]])

In [1]:
from model import EmbeddingLayer

embed = EmbeddingLayer(num_embeddings=20, embedding_dim=512, padding_idx=0, layer_mode='embedding')
embed

EmbeddingLayer(
  (embedding): Embedding(20, 512, padding_idx=0)
  (linear): Embedding(512, 20, padding_idx=0)
)

In [6]:
outputs = embed(inputs)
outputs.shape

torch.Size([64, 20, 512])

In [25]:
import numpy as np
np.sqrt(3)

1.7320508075688772

In [1]:
from model import *
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torch.nn.functional as F
from tokenizer import Tokenizer
from datasets import Dataset
num_embeddings=16000
embedding_dim=512
padding_idx=0
model_dim=512
q_dim=64
k_dim=64
v_dim=64
n_heads=8
in_dim=512
hid_dim=2048
out_dim=512
max_seq_len=20
N=6
mask=True

tokenizer = Tokenizer().load_model()
dataset = Dataset(src='de', trg='en', data_type='train', tokenizer=tokenizer, max_seq_len=20)
train_loader = DataLoader(dataset=dataset, batch_size=64, shuffle=False, collate_fn=dataset.collate_fn)
embed = EmbeddingLayer(num_embeddings=16000, embedding_dim=512, padding_idx=0, layer_mode='embedding')
encoder = Encoder(num_embeddings, embedding_dim, padding_idx, model_dim, q_dim, k_dim, v_dim, n_heads, in_dim, hid_dim, out_dim, max_seq_len, N)
pe = PositionalEncodingLayer()
mha = MultiHeadAttentionLayer(model_dim, q_dim, k_dim, v_dim, n_heads)
decoder = Decoder(num_embeddings, embedding_dim, padding_idx, model_dim, q_dim, k_dim, v_dim, n_heads, in_dim, hid_dim, out_dim, max_seq_len, N, mask)
decoderlayer = DecoderLayer(model_dim, q_dim, k_dim, v_dim, n_heads, in_dim, hid_dim, out_dim, mask)
linear = EmbeddingLayer(num_embeddings=16000, embedding_dim=512, padding_idx=0, layer_mode='linear')
transformer = Transformer(num_embeddings=16000, embedding_dim=512, padding_idx=0, 
                 model_dim=512, q_dim=64, k_dim=64, v_dim=64, n_heads=8,
                 in_dim=512, hid_dim=2048, out_dim=512, max_seq_len=20, N=6)

In [3]:
for epoch in range(1):
    for i, data in enumerate(train_loader):
        inputs, outputs = data
        print("input shape: ", inputs.shape)
        temp = transformer(inputs, outputs)
        print("output shape: ", temp.shape)
        print(temp)

        break

input shape:  torch.Size([64, 20])
output shape:  torch.Size([64, 20, 16000])
tensor([[[5.5684e-05, 6.3741e-05, 1.3426e-04,  ..., 5.6626e-05,
          1.1776e-04, 3.3381e-05],
         [4.9182e-05, 4.0601e-05, 7.3506e-05,  ..., 5.9221e-05,
          9.9536e-05, 1.8590e-05],
         [1.0549e-04, 2.9490e-05, 5.7124e-05,  ..., 3.6819e-05,
          8.1891e-05, 4.9782e-05],
         ...,
         [8.6015e-05, 4.1372e-05, 5.4584e-05,  ..., 6.0921e-05,
          5.6543e-05, 2.2660e-05],
         [9.8022e-05, 6.2315e-05, 2.9807e-05,  ..., 4.3203e-05,
          9.1818e-05, 5.1485e-05],
         [1.2017e-04, 2.0050e-05, 6.4837e-05,  ..., 7.4174e-05,
          6.3432e-05, 4.1033e-05]],

        [[4.8678e-05, 2.1104e-05, 4.1955e-05,  ..., 7.2610e-05,
          4.0270e-05, 3.9668e-05],
         [4.7566e-05, 4.2307e-05, 8.1006e-05,  ..., 7.5113e-05,
          4.9947e-05, 4.7562e-05],
         [3.6857e-05, 5.0062e-05, 8.4756e-05,  ..., 4.3505e-05,
          7.5156e-05, 6.5367e-05],
         ...,
 